# Steps to follow

- clome the mmdetection repo: https://github.com/open-mmlab/mmdetection
- pip install -e . in the mmdetection directory
- Change the file: mmdetection/configs/htc/htc_r50_fpn_1x_artifact.py


python tools/train.py configs/htc/htc_r50_fpn_1x_artifact.py --cfg-options device=mps


In [3]:
from pycocotools.coco import COCO

# Paths to your COCO annotation files
train_anno = '/Users/jbm/Documents/DSAN_6500/WatchdogAI/data_artifacts/train/_annotations.coco.json'
valid_anno = '/Users/jbm/Documents/DSAN_6500/WatchdogAI/data_artifacts/valid/_annotations.coco.json'
test_anno = '/Users/jbm/Documents/DSAN_6500/WatchdogAI/data_artifacts/test/_annotations.coco.json'

# Load each dataset
coco_train = COCO(train_anno)
coco_valid = COCO(valid_anno)
coco_test = COCO(test_anno)

# Print number of images
print(f"Train images: {len(coco_train.getImgIds())}")
print(f"Validation images: {len(coco_valid.getImgIds())}")
print(f"Test images: {len(coco_test.getImgIds())}")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Train images: 69
Validation images: 25
Test images: 10


In [2]:
import json
import random
from pathlib import Path

# Paths
base_dir = Path('/Users/jbm/Documents/DSAN_6500/WatchdogAI/data_artifacts/')
train_path = base_dir / 'train/_annotations.coco.json'
valid_path = base_dir / 'valid/_annotations.coco.json'
test_path  = base_dir / 'test/_annotations.coco.json'

# Load annotation files
with open(train_path) as f:
    train_data = json.load(f)
with open(valid_path) as f:
    valid_data = json.load(f)
with open(test_path) as f:
    test_data = json.load(f)

# Shuffle and split test images
random.seed(42)
random.shuffle(test_data['images'])

test_images = test_data['images'][:10]
extra_images = test_data['images'][10:]

# Get image ids to move
extra_ids = {img['id'] for img in extra_images}
test_ids = {img['id'] for img in test_images}

# Separate corresponding annotations
extra_annotations = [ann for ann in test_data['annotations'] if ann['image_id'] in extra_ids]
test_annotations  = [ann for ann in test_data['annotations'] if ann['image_id'] in test_ids]

# Split the extra images between train and val
extra_val = extra_images[:6]
extra_train = extra_images[6:]

extra_val_ids = {img['id'] for img in extra_val}
extra_train_ids = {img['id'] for img in extra_train}

extra_val_annotations = [ann for ann in extra_annotations if ann['image_id'] in extra_val_ids]
extra_train_annotations = [ann for ann in extra_annotations if ann['image_id'] in extra_train_ids]

# Update original files
train_data['images'].extend(extra_train)
train_data['annotations'].extend(extra_train_annotations)

valid_data['images'].extend(extra_val)
valid_data['annotations'].extend(extra_val_annotations)

test_data['images'] = test_images
test_data['annotations'] = test_annotations

# Save back to disk
with open(train_path, 'w') as f:
    json.dump(train_data, f)

with open(valid_path, 'w') as f:
    json.dump(valid_data, f)

with open(test_path, 'w') as f:
    json.dump(test_data, f)

print("✅ Split complete. Test now has 10 images. Others redistributed to train and valid.")

✅ Split complete. Test now has 10 images. Others redistributed to train and valid.
